<a href="https://colab.research.google.com/github/DarkEol/AutoML-Framework/blob/main/AutoMulti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install auto-sklearn

In [ ]:
!pip install h2o

In [ ]:
#from __future__ import annotations
import autosklearn.classification
import sklearn.model_selection
import h2o
import threading
import time
#import numpy as np
import pandas as pd
#import tensorflow as tf
#import autokeras as ak
from sklearn.datasets import fetch_openml
from sklearn import preprocessing
#from autosklearn.metrics import balanced_accuracy, precision, recall, f1
#from tpot import TPOTClassifier

In [ ]:
class Info:
    def __init__(self, data, target, frame, feature_names):
        self.data = data
        self.target = target
        self.frame = frame
        self.feature_names = feature_names

def runAutoSki(dataset, time, ratio):
  if __name__ == '__main__':
    print("running AutoSklearn")
    X = dataset.data
    y = dataset.target

    if y.dtype=='category':
      print("categorial class encoding")
      #encoding for text values!!!
      le = preprocessing.LabelEncoder()
      y = le.fit_transform(y)

    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=ratio, random_state=1)
    cls = autosklearn.classification.AutoSklearnClassifier(ensemble_class=None, time_left_for_this_task=time)
    cls.fit(X_train, y_train)
    predictions = cls.predict(X_test)
    results.at[0,'AutoML library'] = "AutoSklearn"
    results.at[0,'Algorithm'] = cls.leaderboard().iat[0,2]
    results.at[0,'Accuracy'] = cls.score(X_test, y_test)
    
    """
    #binary classification
    results.at[0,'Precision'] = sklearn.metrics.precision_score(y_test, predictions, pos_label=4)
    results.at[0,'Recall'] = sklearn.metrics.recall_score(y_test, predictions, pos_label=4)
    results.at[0, 'F-measure'] = sklearn.metrics.f1_score(y_test, predictions, pos_label=4)
    results.at[0,'AUC'] = sklearn.metrics.roc_auc_score(y_test, predictions, multi_class='ovr')
    """
    #print("Precision", sklearn.metrics.precision_score(y_test, predictions, average='macro')) #macro for multiclass
    #print("Recall", sklearn.metrics.recall_score(y_test, predictions, average='macro')) #macro for multiclass
    #print("F-measure", sklearn.metrics.f1_score(y_test, predictions, average='macro'))
    pred_proba = cls.predict_proba(X_test)  # for muliclass tasks
    #print("AUC", sklearn.metrics.roc_auc_score(y_test, pred_proba, multi_class='ovr')) #pred_proba for multiclass
    results.at[0,'Precision'] = sklearn.metrics.precision_score(y_test, predictions, average='macro')
    results.at[0,'Recall'] = sklearn.metrics.recall_score(y_test, predictions, average='macro')
    results.at[0, 'F-measure'] = sklearn.metrics.f1_score(y_test, predictions, average='macro')
    results.at[0,'AUC'] = sklearn.metrics.roc_auc_score(y_test, pred_proba, multi_class='ovr')
    
def runH2O(dataset, time, ratio, target_name):
  print("running H2O")
  h2o.init()
  x=dataset.feature_names
  y=dataset.target.name
  frame = h2o.H2OFrame(dataset.frame)
  frame[y] = frame[y].asfactor()
  train, test = frame.split_frame(ratios=[ratio])
  automl = h2o.automl.H2OAutoML(max_runtime_secs=time)
  automl.train(x=x, y=y, training_frame=train)
  perf = automl.leader.model_performance(test)
  perf = automl.leader.model_performance(test)
  print("perf type:", type(perf))
  
  results.at[1,'AutoML library'] = "H2O"
  results.at[1,'Algorithm'] = automl.leader.algo
  """
  #binary
  results.at[1,'Accuracy'] = perf.accuracy()[0][1]
  results.at[1,'Precision'] = perf.precision()[0][1]
  results.at[1,'Recall'] = perf.recall()[0][1]
  results.at[1,'F-measure'] = perf.F1()[0][1]
  results.at[1,'AUC'] = perf.auc()  #[0][1]
  """
  #multiclass
  y_test = test[y].as_data_frame() #pop(0) if target is first column!!
  y_pred = automl.leader.predict(test)['predict'].as_data_frame()
  results.at[1,'Accuracy'] = sklearn.metrics.accuracy_score(y_test, y_pred)
  results.at[1,'Precision'] = sklearn.metrics.precision_score(y_test, y_pred, average='macro', zero_division=0)
  results.at[1,'Recall'] = sklearn.metrics.recall_score(y_test, y_pred, average='macro', zero_division=0)
  results.at[1,'F-measure'] = sklearn.metrics.f1_score(y_test, y_pred, average='macro')
  results.at[1,'AUC'] = sklearn.metrics.roc_auc_score(y_test, y_pred)

#  print("Accuracy score", sklearn.metrics.accuracy_score(y_test, y_pred))
#co_ma = perf.confusion_matrix().cell_values #accuracy multiclass
#print("Accuracy co_ma:", 1 - float(co_ma[len(co_ma[0])-2][len(co_ma[0])-2])) #accuracy multiclass
#print("Precision", sklearn.metrics.precision_score(y_test, y_pred, average='macro', zero_division=0))
#print("Recall", sklearn.metrics.recall_score(y_test, y_pred, average='macro', zero_division=0))
#print("F-measure", sklearn.metrics.f1_score(y_test, y_pred, average='macro'))
#print("AUC", sklearn.metrics.roc_auc_score(y_test, y_pred))

"""
def runAutoKeras(dataset, time, ratio):
  print("running AutoKeras")
  X = dataset.data
  y = dataset.target
  #oe = preprocessing.OrdinalEncoder()
  #X = oe.fit_transform(X)
  le = preprocessing.LabelEncoder()
  y = le.fit_transform(y)
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=ratio, random_state=1)
  clf = ak.StructuredDataClassifier(max_trials=1)
  # Feed the structured data classifier with training data.
  clf.fit(X_train, y_train)
  # Predict with the best model.
  predictions = clf.predict(X_test) 
  results.at[2,'AutoML library'] = "AutoKeras"
  results.at[2,'Accuracy'] = sklearn.metrics.accuracy_score(y_test, predictions)
  results.at[2,'Precision'] = sklearn.metrics.precision_score(y_test, predictions)
  results.at[2,'Recall'] = sklearn.metrics.recall_score(y_test, predictions)
  results.at[2, 'F-measure'] = sklearn.metrics.f1_score(y_test, predictions)
  results.at[2,'AUC'] = sklearn.metrics.roc_auc_score(y_test, predictions)
"""

def runAutoML(dataset, time, ratio):
  print("starting AutoML")
  if type(dataset) == int:
    print("numeric. fetch by id")
    opml = fetch_openml(data_id=dataset, as_frame=True)
  else:
    print("text. fetch from file")
    #load dataset from file
    info = pd.read_csv(dataset, delimiter=';')
    #info = pd.read_csv(dataset)
    y = info["PerformanceRating"] # info[info.columns[info.columns.size-1]]
    X = info.drop("PerformanceRating", axis=1) #info[info.columns.delete(info.columns.size-1)]
    #print("type: ", type(info))
    print("X: ", X)
    print("y: ", y)
    #print("info columns:", info.columns.drop("PerformanceRating"))
    feature_names = list(info.columns.drop("PerformanceRating"))
    target_name = "PerformanceRating"
    opml = Info(X, y, info, feature_names)
  target_name = "PerformanceRating"
  
  #print("feature names: ", opml.feature_names)
  #print("target name: ", type(opml.target))
  # creating thread
  t1 = threading.Thread(target=runAutoSki, args=(opml, time, 1-ratio))
  t1.start()

  # creating thread
  t2 = threading.Thread(target=runH2O, args=(opml, time, ratio, target_name))
  t2.start()

  # creating thread
  #t3 = threading.Thread(target=runAutoKeras, args=(opml, time, ratio))
  #t3.start()

  # wait until threads are completely executed
  t1.join()
  t2.join()
  #t3.join()
  print("results: ", results)
  

results = pd.DataFrame(columns=["AutoML library", "Algorithm", "Accuracy", "Precision", "Recall", "F-measure", "AUC"])

start_time = time.time()
#runAutoML("IBM-Employee3.csv", 300, 0.75)
runAutoML("INX2.csv", 7200, 0.75)
#runAutoML(1590, 300, 0.75)
elapsed = time.time() - start_time
print("elapsed: ", elapsed)

starting AutoML
text. fetch from file
X:        Age  Gender EducationBackground MaritalStatus    EmpDepartment  \
0      32    Male           Marketing        Single            Sales   
1      47    Male           Marketing        Single            Sales   
2      40    Male       Life Sciences       Married            Sales   
3      41    Male     Human Resources      Divorced  Human Resources   
4      60    Male           Marketing        Single            Sales   
...   ...     ...                 ...           ...              ...   
1195   27  Female             Medical      Divorced            Sales   
1196   37    Male       Life Sciences        Single      Development   
1197   50    Male             Medical       Married      Development   
1198   34  Female             Medical        Single     Data Science   
1199   24  Female       Life Sciences        Single            Sales   

            EmpJobRole BusinessTravelFrequency  DistanceFromHome  \
0      Sales Executive   

/usr/local/lib/python3.9/dist-packages/autosklearn/data/feature_validator.py:318: UserWarning: Input Column Gender has generic type object. Autosklearn will treat this column as string. Please ensure that this setting is suitable for your task.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/autosklearn/data/feature_validator.py:318: UserWarning: Input Column EducationBackground has generic type object. Autosklearn will treat this column as string. Please ensure that this setting is suitable for your task.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/autosklearn/data/feature_validator.py:318: UserWarning: Input Column MaritalStatus has generic type object. Autosklearn will treat this column as string. Please ensure that this setting is suitable for your task.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/autosklearn/data/feature_validator.py:318: UserWarning: Input Column EmpDepartment has generic type object. Autosklearn will treat this column as string. P

... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.18" 2023-01-17; OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1); OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.9/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp_l1hyeeb
  JVM stdout: /tmp/tmp_l1hyeeb/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp_l1hyeeb/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.3
H2O_cluster_version_age:,6 days
H2O_cluster_name:,H2O_from_python_unknownUser_2lxuif
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |██████████████████████████████████████████████████████████████[WARNING] [2023-04-10 20:21:54,692:Client-AutoML(1):958c4262-d7cc-11ed-808b-0242ac1c000c] No valid ensemble was created. Please check the logfile for errors. Default to the best individual estimator:(1, 134, 0.0)
RunKey(config_id=1, instance_id='{"task_id": "958c4262-d7cc-11ed-808b-0242ac1c000c"}', seed=0, budget=0.0) RunValue(cost=0.21212121212121215, time=6.419314861297607, status=<StatusType.SUCCESS: 1>, starttime=1681150925.4189987, endtime=1681150931.8879466, additional_info={'duration': 6.111483573913574, 'num_run': 2, 'train_loss': 0.0, 'configuration_origin': 'Initial design'})
RunKey(config_id=2, instance_id='{"task_id": "958c4262-d7cc-11ed-808b-0242ac1c000c"}', seed=0, budget=0.0) RunValue(cost=0.08754208754208759, time=11.177189111709595, status=<StatusType.SUCCESS: 1>, starttime=1681150931.8942933, end

Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-6-c5822c163504>", line 76, in runH2O
  File "/usr/local/lib/python3.9/dist-packages/sklearn/utils/validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_ranking.py", line 536, in roc_auc_score
    raise ValueError("multi_class must be in ('ovo', 'ovr')")
ValueError: multi_class must be in ('ovo', 'ovr')


results:    AutoML library          Algorithm  Accuracy Precision    Recall F-measure  \
0    AutoSklearn  gradient_boosting  0.906667  0.860614  0.870753  0.865597   
1            H2O            xgboost  0.936455  0.931936  0.897905  0.911067   

        AUC  
0  0.964084  
1       NaN  
elapsed:  7237.136662721634


In [ ]:
results.to_excel('Experiments.xlsx')